---
title: Inference and Optimization
---





## Learning Objectives {.unnumbered}

## Optimization: calculus perspective

## Optimization: overview of methods

## Curse of dimensionality

## Maximum Likelihood Estimation (MLE)

We have some *parametric statistical model* with unknown parameters.
We want to evaluate how consistent the data are with different values of the parameters, and to find the values of the parameters that are most consistent with the data.

### Likelihood function for one observation

The likelihood is the probability of the **data** given some **parameters**:
$$
p(y | \theta)
$$

Often, we want to study how the likelihood changes for different values of $\theta$, holding $y$ fixed.
This is just $p(y | \theta)$ for a range of $\theta$.


::: {.callout-note}
You will sometimes see this referred to as $\mathcal{L}(\theta)$, which is a  confusing notation...
:::

### Likelihood function for multiple observations

Independent and identically distributed (i.i.d.) assumption:
$$
\begin{align}
p(y_1, y_2, \ldots, y_n) &= p(y_1) p(y_2) \times \ldots \times p(y_n)\\
 &= \prod_{i=1}^n p(y_i)
\end{align}
$$


Usually we have more than one data point.
Say we measure $y = y_1, y_2, \ldots, y_n$:
$$
p(y | \theta) = \prod_{i=1}^n p(y_i | \theta)
$$

## Log trick

Recall: $\log(AB) = \log(A) + \log(B)$ or, more generally,
$$
\log \left( \prod_{i=1}^n f_i \right) = \sum_{i=1}^n \log(f_i)
$$

Thus, we can work with the "log likelihood":
$$
\log p(y | \theta) =  \log \left( \prod_{i=1}^n p(y_i | \theta) \right) = \sum_{i=1}^n \log \left( p(y_i | \theta) \right)
$$

Adding small numbers is also more numerically stable than multiplying them

### Maximum likelihood estimation

## Regression models (inference)

### GLMs

## Old


Can we find the parameters $\theta^*$ that maximize the likelihood $p(y | \theta)$?

## Log likelihood

We can use the log likelihood $\log p(y | \theta)$ instead of the likelihood $p(y | \theta)$.

The log likelihood is monotonic with the likelihood, so 
$$
\arg \max \log p(y | \theta) = \arg \max p(y | \theta)
$$

## Analytic solution {.scrollable .smaller}

Solving things analytically takes time up-front, but can be much faster to run because you can avoid the optimization step.
Consider the (potentially multivariate) Gaussian example with known covariance matrix $\Sigma$.
We want to *maximize* the likelihood
$$
\sum_{i=1}^n p(y_i | \mu, \Sigma)
$$

. . .

To maximize, we set its derivative with respect to $\mu$, which we'll denote with $\nabla_\mu$, to zero:
$$
\sum_{i=1}^n \nabla_\mu \log p(y_i | \mu, \Sigma) = 0
$$

. . .

Substituting in the multivariate Gaussian likelihood we get:
$$
\begin{aligned}
0 & =\sum_{i=1}^n \nabla_\mu \log \frac{1}{\sqrt{(2 \pi)^d|\Sigma|}} \exp \left(-\frac{1}{2}\left(x_i-\mu\right)^{\top} \Sigma^{-1}\left(x_i-\mu\right)\right) \\
& =\sum_{i=1}^n \nabla_\mu\left(\log \left(\frac{1}{\sqrt{(2 \pi)^d|\Sigma|}}\right)\right)+\log \left(\exp \left(-\frac{1}{2}\left(x_i-\mu\right)^{\top} \Sigma^{-1}\left(x_i-\mu\right)\right)\right) \\
& =\sum_{i=1}^n \nabla_\mu\left(-\frac{1}{2}\left(x_i-\mu\right)^{\top} \Sigma^{-1}\left(x_i-\mu\right)\right)\\
&=\sum_{i=1}^n \Sigma^{-1}\left(x_i-\mu\right) \\
0 &= \sum_{i=1}^n (x_i - \mu) \\
\mu &= \frac{1}{n} \sum_{i=1}^n x_i
\end{aligned}
$$

. . .

::: {.callout-note}
You are not expected to remember the above equations and I won't ask you to do this derivation in a time-constrained exam.
You should understand the general procedure:

1. write down log likelihood for all data points
    1. write down likelihood for one data point
    1. write down log likelihood for one data point
    1. sum over all data points
1. take $\frac{d}{d\theta}$ and set equal to zero to maximize
1. solve for $\theta^*$.

:::

## Numerical approach I

We can use the `optimize` function from the `Optim.jl` package to find the maximum likelihood estimate.
First, we need to define the function to be optimized.
`optimize` will minimize the function, so we need to define the *negative* log likelihood.
We'll call this the "loss" function.


In [ ]:
loss(θ) = -normal_log_lik(y_multi, θ[1], θ[2]); # <1>

1. Note that this function takes in **a single argument** which is a vector of parameters. We'll call this vector `θ` but it doesn't matter what we call it.

## Numerical approach II {.scrollable .smaller}

Now we can run the optimization.
Since $\sigma > 0$ always, we will pass along bounds.
We could alternatively do something clever like work with $\log \sigma$ instead of $\sigma$.


In [ ]:
lower = [0.0001, 0.0001] # <1>
upper = [Inf, Inf] # <2>
guess = [1.0, 1.0] # <3>

res = optimize(loss, lower, upper, guess) # <4>
θ_MLE = Optim.minimizer(res) # <5>

1. The lower bound is actually zero, but we just set it to a "pretty small" number.
2. The upper bound is infinity, we can pass in `Inf`
3. We need to pass in a guess for the parameters. We'll just use $\mu = \sigma = 1$.
4. This will actually run the optimization
5. This will extract the parameters that minimize the loss function.

We could convert this to a `Distributions` object as


In [ ]:
dist_MLE = Normal(θ_MLE[1], θ_MLE[2])

# Regression example

## Overview {.scrollable}

Let's consider the generic regression probelem where we have paired observations $\left\{x_i, y_i\right\}_{i=1}^n$.
In general, we can write this regression as
$$
y_i | \alpha, \beta, \epsilon \sim \mathcal{N}(\alpha + x_i \beta, \sigma^2)
$$
where $x_i$ and $\beta$ may be vectors.

. . .

We can create some raw data (click to "unfold" the code)


In [ ]:
#| code-fold: true
X = [
    9.4,
    11.2,
    18.5,
    5.7,
    6.4,
    4.4,
    10.3,
    16.0,
    7.8,
    12.2,
    12.3,
    15.1,
    10.3,
    12.4,
    8.2,
    11.5,
    9.0,
    11.3,
    9.4,
    8.5,
]
y = [
    19.4,
    25.0,
    41.6,
    11.9,
    10.6,
    8.0,
    21.8,
    33.8,
    15.4,
    24.9,
    27.4,
    31.8,
    18.6,
    30.1,
    18.1,
    25.8,
    18.8,
    24.0,
    17.4,
    14.7,
]
scatter(X, y; xlabel=L"$X$", ylabel=L"$y$", label="")

## Anlytic approach

We can use the same approach to derive the maximum likelihood estimate for linear regression:

1. Write the likelihood for one data point
1. Write the log likelihood for one data point
1. Write the log likelihood for all data points
1. Take $\frac{d}{d\theta}$ and set equal to zero to maximize

If you want a walkthrough, see [Ryan Adams's lecture notes](https://www.cs.princeton.edu/courses/archive/fall18/cos324/files/mle-regression.pdf) starting at about equation 11.

## Numerical optimization I

As before, we need to write down a (log) likelihood function


In [ ]:
function reg_loglik(xi::T, yi::T, α::T, β::T, σ::T) where {T<:Real}
    μ = α + xi * β
    dist = Normal(μ, σ)
    return logpdf(dist, yi)
end;

. . .


In [ ]:
function reg_loglik(X::Vector{T}, y::Vector{T}, α::T, β::T, σ::T) where {T<:Real}
    return sum([reg_loglik(xi, yi, α, β, σ) for (xi, yi) in zip(X, y)])
end;

## Numerical optimization II


In [ ]:
loss(θ) = -reg_loglik(X, y, θ[1], θ[2], θ[3]); # <1>
lower = [-Inf, -Inf, 0.0001] # <2>
upper = [Inf, Inf, Inf]
guess = [1.0, 1.0, 1.0]
res = optimize(loss, lower, upper, guess)
round.(Optim.minimizer(res); digits=3) # <3>

1. This is the loss function, which is the negative log likelihood. The first value of $\theta$ is the intercept, the second is the slope, and the third is the standard deviation.
2. We need to pass in bounds for the parameters. The standard deviation is always positive, so we set the lower bound to a small number.
3. This will extract the parameters that minimize the loss function and round to show three decimal places.

## Parallel: least squares {.scrollable .smaller}

If we work through the math, we can show that the log likelihood for the linear regression problem is
$$
\log p(y | X, \beta, \sigma) = \frac{N}{2} \log (2 \sigma^2 \pi)  - \frac{1}{2 \sigma^2} \left( X \beta - y \right)^T \left( X \beta - y \right)
$$

::: {.callout-note}
## Linear algebra notation

There is no intercept here!
This is a common notation and assumes that the first column of $X$ is all ones.
That is equivalent to writing down an intercept, but lets us use linear algebra notation and keep track of fewer variable names
:::

. . .

From this, we can show that terms drop out and
$$
\beta^\text{MLE} = \arg \min_\beta \left( X \beta - y \right)^T \left( X \beta - y \right)
$$
which is exactly the least squares problem (minimize squared error):
$$
\min_{\theta} \sum_{i=1}^n (y_i - y_i^\text{pred})^2
$$

. . .

::: {.callout-important}
## Key point

"Least squares can be interpreted as assuming Gaussian noise, and particular choices of likelihood can be interpreted directly as (usually exponentiated) loss functions" --[Adams](https://www.cs.princeton.edu/courses/archive/fall18/cos324/files/mle-regression.pdf)
:::

If we then want to estimate $\sigma$, we can estimate the standard deviation of the residuals.

# Wrapup

## Don't get it twisted

::: {.note}
Many people get this backwards!
:::

::: {.incremental}

- The likelihood is the probability of the data given the parameters: $p(y | \theta)$.
- We often plot the likelihood for many different $\theta$
    - $p(y | \theta)$ for many different $\theta$
- Don't confuse this with the posterior, which is the probability of the parameters given the data: $p(\theta | y)$

:::

## Logistics

- Friday:
    - Lab 03 in class -- look for GH Classroom link on Canvas
    - Lab 02 due
- Next week:
    - Bayesian inference

## References